In [ ]:
# import package

import urllib.request
from bs4 import BeautifulSoup
import os
import time
from tqdm import tqdm
import re
import pandas as pd

folder_path = '組員的電腦路徑'
if os.path.exists(folder_path) == False:
    os.makedirs(folder_path)


def crawl_page(url):
    mainUrl = 'http://www.7k7k.net.cn/star/'
    targetUrl = mainUrl + url
    headers = {
        'User-Agent': '組員電腦的'}
    web = requests.get(targetUrl, headers=headers)
    return web.content


def crawl_all(n):
    page_lst = []
    for i in range(1, n):
        if i == 1:
            nPage = 'index.html'
            page_lst.append(nPage)
        else:
            nPage = 'index_%d.html' % (i)
            page_lst.append(nPage)

    pageNum = 0
    index_lst = []
    name_lst = []
    info_lst = []
    for url in tqdm(page_lst):
        web = crawl_page(url)
        soup = BeautifulSoup(web, 'html.parser')
        items = soup.find_all('img')
        items = items[1:]
        if pageNum % 10 == 0:
            time.sleep(5)
        for index, item in enumerate(items):
            if item:
                try:
                    html = requests.get(item.get('src'))
                    img_name = folder_path + str(index + 25*pageNum) + '.png'
                    with open(img_name, 'wb') as file:
                        file.write(html.content)
                        file.flush()
                    file.close()
                    print('第%d張圖片下載完成' % (index + 25*pageNum))
                    time.sleep(1)
                except:
                    pass

        names = soup.find_all('a')
        names = names[20:70]
        for index, name in enumerate(names):
            if name:
                html = requests.get(
                    'http://www.7k7k.net.cn' + name.get('href'))
                html.encoding = 'html'
                pattern_name = re.compile('itemprop="name">(.*?)<\/b>')
                pattern_info = re.compile(
                    'class="w200"><span>身　　高：<\/span>(.*?)CM<\/p>\s+<p.*?><span>体　　重：<\/span>(.*?)KG<\/p>')
                popName = re.findall(pattern_name, html.text)
                popInfo = re.findall(pattern_info, html.text)
                index_lst.append(index + 25*pageNum)
                name_lst.append(popName[0])
                if len(popInfo) == 0:
                    info_lst.append('NA')
                else:
                    info_lst.append(popInfo[0])
        pageNum += 1

    return index_lst, name_lst, info_lst


index_lst, name_lst, info_lst = crawl_all(143)


Index_lst = index_lst[:3550]
Name_lst = []
for i in range(0, len(name_lst), 2):
    Name_lst.append(name_lst[i])
Info_lst = []
for i in range(0, len(info_lst), 2):
    Info_lst.append(info_lst[i])

data = {'index': [item for item in Index_lst],
        'name': [item for item in Name_lst],
        'height': [item[0] for item in Info_lst],
        'weight': [item[1] for item in Info_lst]}
data = pd.DataFrame(data)

outputpath = '組員的電腦路徑'
data.to_csv(outputpath, index=False, header=True)